In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [4]:
!pip install transformers==4.3.3
!pip install stop_words
!pip install symspellpy
!pip install language_detector 
!pip install cached_property
!pip install sentencepiece
!pip install config
!pip install umap
!pip install sentence-transformers==0.4.1
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
!pip install tensorflow==2.

In [3]:
# root_path = "/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2"
import sys, os
import config

config.root_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
sys.path.insert(0, config.root_path)

In [4]:
import tensorflow as tf

from tensorflow.python import keras
import json
import pandas as pd
import numpy as np
import os

from src.encoders.context_encoder_ldabert_2 import ContextEncoderComplex, ContextEncoderSimple
from src.dataset.ldabert_2 import LDABERT2Dataset
import transformers
import sentence_transformers

from utils.metrics import windowdiff, pk

Downloading:   0%|          | 0.00/630 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/409 [00:00<?, ?B/s]

In [5]:
tf.__version__, transformers.__version__, sentence_transformers.__version__

('2.7.0', '4.3.3', '0.4.1')

## LDA BERT 2 Dataset

In [8]:
dataset_type_to = "clinical"
dataset = LDABERT2Dataset(dataset_type=dataset_type_to,
                           pct_data=0.075,
                          max_seq_length=128,
                           max_segment_length=300,
                           augment_pct=0)

sentences, tokenized_sentences, labels = dataset.process()

# vectors_path = '../data/clinical_vectors/lda_bert_{}_{}.pkl'.format(dataset_type, len(sentences))
vectors_filename = '{}_{}.pkl'.format(dataset.pct_data, dataset.augment_pct)

saved_vectors, saved_labels, saved_tokens, saved_tokenized_sentences = dataset.get_saved_vectors("test", dataset.dataset_type, vectors_filename)

if len(saved_vectors) == 0:
    saved_vectors, saved_labels, saved_tokens, saved_tokenized_sentences = dataset.create_vectors("test", dataset.dataset_type, vectors_filename)

artificial segments False
artificial segments False


ImportError: Unable to convert output to TensorFlow tensors format, TensorFlow is not installed.

In [7]:
left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(saved_tokenized_sentences)

NameError: name 'saved_tokenized_sentences' is not defined

In [9]:
lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(saved_vectors)

In [10]:
saved_tokenized_sentences['input_ids'].shape, saved_vectors.shape

(TensorShape([2285, 128]), (2285, 12))

In [11]:
saved_tokens

['The arterial pulse changes as it travels from the central aorta down to the peripheral arteries ',
 'Although mean blood pressure decreases from the central aorta to the peripheral arteries  the systolic pressure increases ',
 'The pulse can be distorted and damped by reflected  resonance  or standing waves  as well as differences in the elastic properties and caliber of the peripheral arteries of the upper and lower parts of the body ',
 'The size of the normal arterial pulse is affected by the left ventricular ejection volume and the rate of ejection  as well as the elasticity or distensibility of the peripheral arteries ',
 'The greater the distance from the heart to a peripheral artery  the greater will be the distortion of the arterial pulse peripherally ',
 'Because complex factors affect the contour of the arterial pulse as it is transmitted through the arterial system  careful examination of the most central arterial pulse  the carotid  is necessary for pulse contour informat

## Model

In [32]:
lda_gamma = 15
model = ContextEncoderComplex(final_dropout=0.5,
                            dense_neurons=256,
                                  lstm_size=256,
                             lstm_dropout_percentage=0.5,
                             cnn_filters=8,
                             cnn_kernel_size=3,
                             pool_size=2,
                          gamma=lda_gamma,
                          max_sentence_length=128,
                           bert_trainable=False)
# model = ContextEncoderSimple(final_dropout=0.5,
#                             dense_neurons=256,
#                           gamma=lda_gamma,
#                           max_sentence_length=128,
#                            bert_trainable=False)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [33]:
hash = "9Udxa"
num_samples = "37412"
dataset_type_from = "wiki"
model_path = 'LDABERT2/complex/{}-{}-1-pct-1-aug_{}/no-finetune'.format(dataset_type_from, num_samples, hash)
checkpoint_filepath = '../models/{}/checkpoint'.format(model_path)
log_file = '../predictions/{}_lda-{}-{}/predictions.csv'.format(hash, lda_gamma, dataset_type_to)

In [34]:
# try:
model.load_weights(checkpoint_filepath)
# except:
#     print("No checkpoint available.")

In [36]:
model.summary()

Model: "context_encoder_complex_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tf_bert_model_2 (TFBertMode  multiple                 109482240 
 l)                                                              
                                                                 
 bi-directional_left (Bidire  multiple                 542720    
 ctional)                                                        
                                                                 
 bi-directional_mid (Bidirec  multiple                 542720    
 tional)                                                         
                                                                 
 bi-directional_right (Bidir  multiple                 542720    
 ectional)                                                       
                                                                 
 left_context (Attention)    multiple    

In [15]:
saved_tokenized_sentences

{'input_ids': <tf.Tensor: shape=(2285, 128), dtype=int32, numpy=
array([[  101,  1996, 25543, ...,     0,     0,     0],
       [  101,  2348,  2812, ...,     0,     0,     0],
       [  101,  1996,  8187, ...,     0,     0,     0],
       ...,
       [  101, 21961, 23586, ...,     0,     0,     0],
       [  101,  2035,  1997, ...,     0,     0,     0],
       [  101,  1996,  3019, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2285, 128), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2285, 128), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=in

## Inference

In [35]:
slice_size = 2
predictions_log = []

for threshold in [0.5]:
  for i in range(0, saved_vectors.shape[0]//slice_size):
      start = i*slice_size
      end = start + slice_size
      print(start, end)

      sliced_tokenized_sentences = {
          'input_ids': saved_tokenized_sentences['input_ids'][start:end],
          'token_type_ids': saved_tokenized_sentences['token_type_ids'][start:end],
          'attention_mask': saved_tokenized_sentences['attention_mask'][start:end]
      }
      sliced_vectors = saved_vectors[start:end]
      
      left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(sliced_tokenized_sentences)
      lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(sliced_vectors)
      
      logits = model([
          left_input, 
          mid_input, 
          right_input, 
          lda_left_input, 
          lda_mid_input, 
          lda_right_input
      ])

      logits_flattened = [float(p) for p in logits]

      predictions = [1 if float(logit) >= threshold else 0 for logit in logits_flattened]

      k = 14

      string_predictions = "".join([str(i) for i in predictions])
      string_ground_truth = "".join([str(i) for i in saved_labels[start:end]])
      print(string_predictions)
      print(string_ground_truth)
      overall_windowdiff = windowdiff(string_predictions, string_ground_truth, k)
      overall_pk = pk(string_predictions, string_ground_truth, k)

      for pred_idx, pred in enumerate(predictions):
        log = {
            "prediction": pred,
            "ground_truth": saved_labels[start:end][pred_idx],
            "text": saved_tokens[start:end][pred_idx],
            "lda_gamma": lda_gamma,
            "prediction_threshold": threshold,
            "logit": logits_flattened[pred_idx]
        }
        predictions_log.append(log)

      print("{},{},{},{}".format(overall_windowdiff, overall_pk, k, k))

0 2
sbert_output TFBaseModelOutputWithPooling(last_hidden_state=<tf.Tensor: shape=(2, 128, 768), dtype=float32, numpy=
array([[[-0.03473086, -0.80840105,  1.3095813 , ...,  0.52079695,
          0.46027163,  0.43990484],
        [-0.34313473, -0.8674657 ,  1.2323778 , ...,  0.35141557,
          0.22114596,  0.33552533],
        [ 0.00544309, -0.39362085,  1.1972243 , ...,  0.13438508,
          0.25531107,  0.7349293 ],
        ...,
        [-0.03999893, -0.40749472,  1.1493117 , ...,  0.44348675,
         -0.03826812,  0.31083706],
        [-0.0302548 , -0.3630402 ,  1.0503726 , ...,  0.35737997,
         -0.1469976 ,  0.3225574 ],
        [-0.06537521, -0.4042562 ,  1.0459397 , ...,  0.43760508,
         -0.07244518,  0.31850904]],

       [[ 0.8253576 , -1.1223234 ,  1.0488348 , ..., -0.06074864,
         -0.40136987,  0.6455008 ],
        [ 0.75767094, -1.4008561 ,  1.1932728 , ...,  0.15608771,
         -0.7739211 ,  0.55741656],
        [ 0.90204036, -1.0028046 ,  1.1936677 , ..

ValueError: ignored

In [ ]:
try:
  import csv
  if not os.path.exists(os.path.dirname(log_file)):
      os.makedirs(os.path.dirname(log_file))
  mode = 'a' if os.path.exists(log_file) else 'w'
  with open(log_file, mode) as csvfile:
      writer = csv.DictWriter(csvfile, fieldnames=["prediction", "ground_truth", "text", "lda_gamma", "prediction_threshold", "logit"])
      writer.writeheader()
      for data in predictions_log:
          writer.writerow(data)
except IOError:
    print("I/O error")

In [ ]:
[float(p) for p in predictions], saved_labels

([0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0],
 [1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,